### text boxes appear outside of tabs -- how to add inside tabs?  
### get input from text boxes (signal/slots?)
### pass the dir & set of exps to class that creates h1's

In [ ]:
import sys
from PyQt5.QtWidgets import (QMainWindow, QApplication, QPushButton, QWidget, QAction, 
                             QTabWidget,QVBoxLayout, QHBoxLayout, QInputDialog, QLineEdit, QLabel)
from PyQt5.QtGui import QIcon
from PyQt5.QtCore import pyqtSlot, QCoreApplication

 
class App(QMainWindow):
 
    def __init__(self):
        super().__init__()
        self.title = 'Peak picker'
        self.left = 0
        self.top = 0
        self.width = 600
        self.height = 400
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)
 
        self.table_widget = MyTableWidget(self)
        self.setCentralWidget(self.table_widget)
 
        self.show()
    
class MyTableWidget(QWidget):        
 
    def __init__(self, parent):   
        #super().__init__()
        super(QWidget, self).__init__(parent)
        
 
        # Initialize tab widget
        self.tabs = QTabWidget()
        
        # create tabs 
        self.navTab = QWidget()
        self.tab2 = QWidget()
        
        self.tabs.resize(600,400) 
         
        # create vertical layout for navigation tab 
        self.navLayout = QVBoxLayout(self)
    
        # create rows with text box inputs for directory input 
        self.dirLayout = QHBoxLayout(self)
        self.filesLayout = QHBoxLayout()
        
        # directory 'row'
        self.dirLabel = QLabel('Directory: ')
        self.dirInp = QLineEdit('dirs here')
        
        # add directory 'row' to widget
        self.dirLayout.addWidget(self.dirLabel)
        self.dirLayout.addWidget(self.dirInp)
        
        # add dir widget to navLayout??
        self.navLayout.addLayout(self.dirLayout)
        
    
        # files 'row'
        self.filesLabel = QLabel('Files: ')
        self.filesInp = QLineEdit('files here')
        
        # add files 'row to widget
        self.filesLayout.addWidget(self.filesLabel)
        self.filesLayout.addWidget(self.filesInp)
        
        # add files widget to navLayout?
        self.navLayout.addLayout(self.filesLayout)
        
        # add nav layout to tabs 
        self.navLayout.addWidget(self.tabs)
        
        
        
         # Add tabs to tab widget 
        self.tabs.addTab(self.navTab,"Navigate")
        self.tabs.addTab(self.tab2,"Pick")
        
        
        #self.navTab.setLayout(self.navLayout)
        
        
        #self.setLayout(self.layout)  ### this line might not be necessary??
        
        
        #self.initUI()
        
    def initUI(self):
        self.getText()
 
    #@pyqtSlot()
    #def on_click(self):
        #print("\n")
        #for currentQTableWidgetItem in self.tableWidget.selectedItems():
            #print(currentQTableWidgetItem.row(), currentQTableWidgetItem.column(), currentQTableWidgetItem.text())
            
    def getText(self):
        
        print(self.dirInp.text())

    

if __name__ == '__main__':
    app = QCoreApplication.instance() ### adding this if statement prevents kernel from crashing 
    if app is None:
        app = QApplication(sys.argv)
        print(app)
    ex = App()
    sys.exit(app.exec_())

In [ ]:
import os
import sys
from glob import glob
from collections import defaultdict
from collections import Counter
import collections
import re
import shutil
import subprocess
import pandas as pd

class site_data:
    #get_h1s()
    def __init__(self):
    
        self.ant_set = {'ant'}
        self.ans_set = {'ans'}
        self.other_exps = {'vp3', 'cpt', 'ern', 'aod', 'stp', 'gng'}
        
    #get_h1s()    
    def check_cnt_copy(self, fp_check_cnt, exps_list):
        """
        create dictionary of lists where id is key and exp*cnt names are the values
        """
        self.fp_check_cnt = fp_check_cnt
        self.exps_list = exps_list

        cnt_dict = {}
        for r,d,f in os.walk(fp_check_cnt):
            for n in f:
                if n.endswith('_32.cnt'):
                    split = n.split('_')
                    cnt_dict.setdefault(split[3], []).append(split[0])
                    
        for k,v in cnt_dict.items():
            if len(v) != len(exps_list):
                for missing in exps_list:
                    if missing not in v:
                        print("{} missing from {}'s folder".format(missing, k))
                
    #get_h1s()   --NEED TO CHANGE THIS               
    def rename_reruns(self, fp_rerun, fp_rerun_trg=None):
        """
        if peak-picking, copy rr file to target directory and rename it.
        if not peak-picking, only let me know there is a rr file
        """
        self.fp_rerun = fp_rerun
        self.fp_rerun_trg = fp_rerun_trg
        
        #check for _rr.cnt regardless of run number 
        for r,d,f in os.walk(fp_rerun):
            for n in f:
                if n.endswith('_rr.cnt'):
                    print('Found re-runs...',  os.path.basename(fp_rerun), n)
                    answer=input('Copy to target directory?')
                    if answer in ['y', 'Y', 'yes', 'Yes']:
                        shutil.copy(os.path.join(r,n), os.path.join(fp_rerun_trg, n[:3])) 
                    else:
                        pass
                    
        #rename only if peak picking              
        if fp_rerun_trg:
            for r,d,f in os.walk(fp_rerun_trg):
                for n in f:
                    if n.endswith('_rr.cnt'):
                        new_name = os.rename(os.path.join(r, n), os.path.join(r,n.replace("_rr", "")))
                        print('\n', 'Renaming', os.path.join(r,n), '\n', 
                              'to','\n', os.path.join(r,n.replace("_rr", ""), '\n'))
                        
    #get_h1s()                    
    def create_cnth1(self, fp_cnt):
        """
        .cnt => .cnt.h1
        """
        
        self.fp_cnt = fp_cnt
        
        print('\n', '_________________________________________________CREATING CNT.H1S_________________________________________________')
        for r,d,f in os.walk(fp_cnt):
            for n in f:
                if n.startswith(self.cnth1_tups) and n.endswith('_32.cnt'):
                    path = os.path.join(r,n)
                    p = subprocess.Popen("create_cnthdf1_from_cntneuroX.sh {}".format(path),shell=True, 
                                          stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=os.path.dirname(path))
                    result = p.communicate()[1]
                    print(result.decode('ascii'))
                    
    #get_h1s()                
    def create_avgh1(self, fp_h1):
        """
        .cnt.h1 => .avg.h1
        """
        
        self.fp_h1 = fp_h1

        print('\n', '_________________________________________________CREATING AVG.H1_________________________________________________')            
        for r,d,f in os.walk(fp_h1):
            for n in f:
                if n.startswith(self.ant_tup) and n.endswith('cnt.h1'):
                    ant_path = os.path.join(r,n)
                    p_ant = subprocess.Popen('create_avghdf1_from_cnthdf1X -lpfilter 8 -hpfilter 0.03 -thresh 75 -baseline_times -125 0 {}'.format(ant_path), 
                                             shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=os.path.dirname(ant_path))
                    result_ant = p_ant.communicate()[1]
                    print(result_ant.decode('ascii'))
                if n.startswith(self.ans_tup) and n.endswith('cnt.h1'):
                    ans_path = os.path.join(r,n)
                    p_ans = subprocess.Popen('create_avghdf1_from_cnthdf1X -lpfilter 16 -hpfilter 0.03 -thresh 100 -baseline_times -125 0 {}'.format(ans_path), 
                                            shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=os.path.dirname(ans_path))
                    result_ans = p_ans.communicate()[1]
                    print(result_ans.decode('ascii'))
                if n.startswith((self.others_tup)) and n.endswith('cnt.h1'):
                    path=os.path.join(r,n)
                    p_others = subprocess.Popen('create_avghdf1_from_cnthdf1X -lpfilter 16 -hpfilter 0.03 -thresh 75 -baseline_times -125 0 {}'.format(path), 
                                                shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=os.path.dirname(path))
                    result_others = p_others.communicate()[1]
                    print(result_others.decode('ascii'))
                    
                    
    #get_h1s()
    def create_avgps(self, fp_ps):
        """
        avg.h1 => avg.h1.ps in pwd
        """
        self.fp_ps = fp_ps

        print('\n', '_________________________________________________CREATE H1.PS_________________________________________________')
        
        for path, subdirs, files in os.walk(fp_ps):
            for name in files:
                if name.endswith("avg.h1"):
                    ps_paths = os.path.join(path, name)
                    subprocess.Popen("plot_hdf1_data.sh {}".format(name), shell=True, cwd=os.path.dirname(ps_paths))
                    print("Creating ps files...  {}".format(name))
                    
    #get_h1s()                
    def delete_bad_files(self, fp):
        """
        returns any extension not in exts_to_keep & prompts user to delete 
        """
        self.fp = fp

        print('\n', '_________________________________________________CLEANING UP...________________________________________________')
    
        for r,d,f in os.walk(fp):
            for n in f:
                if n.endswith(('_32.cnt', '_cnt.h1')):
                    os.remove(os.path.join(r,n))
                    print("\nRemoving {}\n".format(n))
                        
    def get_h1s(self, fp, set_of_exps, ps=None, trg_dir=None):
        """
        combines all these commands together
        """
        self.fp = fp
        self.set_of_exps = set_of_exps
        self.ps = ps
        self.trg_dir = trg_dir
        
        #use in create_cnth1()
        self.cnth1_tups = tuple(set_of_exps)
        
        if_ant = self.ant_set & set_of_exps
        if_ans = self.ans_set & set_of_exps
        all_others = self.other_exps & set_of_exps
        
        #used in create_avgh1()
        self.ant_tup = tuple(if_ant)
        self.ans_tup = tuple(if_ans)
        self.others_tup = tuple(all_others)

        #if being used for peak picking, create new directories and move all cnt files to the correct folder...and so on
        if trg_dir:
            #create new directories
            exps_list = list(set_of_exps)
            for exp in exps_list:
                new_dirs = os.path.join(trg_dir, exp)
                if not os.path.exists(new_dirs):
                    os.makedirs(new_dirs)
                    print("Creating {}\n".format(new_dirs))
                else:
                    print("{} already exist\n".format(new_dirs))
            #copy cnt files to newly created directories       
            count = 0
            for r,d,f in os.walk(fp):
                for n in f:
                    if n.startswith(self.cnth1_tups) and n.endswith('_32.cnt'):
                        count+=1
                        print("Copying {}".format(n))
                        shutil.copy(os.path.join(r,n), os.path.join(trg_dir, n[:3]))
                        if count % len(exps_list) == 0:
                            print('\n')
            print("\nCopied {} files.\n".format(count))
                        
            self.rename_reruns(fp,fp_rerun_trg=trg_dir)
            self.check_cnt_copy(trg_dir, exps_list)
            self.create_cnth1(trg_dir)
            self.create_avgh1(trg_dir)
            self.delete_bad_files(trg_dir)
            if ps:
                self.create_avgps(trg_dir)
            return True
        
        #if you want to create avg.h1s IN directory copying files...
        self.check_cnt_copy(trg_dir, exps_list)
        self.rename_reruns(fp)
        self.create_cnth1(fp)
        self.create_avgh1(fp)
        self.delete_bad_files(fp)
        if ps:
            self.create_avgps(fp)
pick = site_data()